In [9]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time

In [10]:
## get problem parameters and geometry
problem = problems.problem3

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F):
    
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*(G/KBTV)* Trace(F.trans*F ) + H


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord-2)

fes = FESpace([fesu, fesphi])
u, psi = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)


## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((1,1,1))

psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0
obs = CF(-z)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, psi, psik = G_funs
    F = Id(3)+ grad(u)
    if form == "Functional":
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx) # add the "bilinear" form of specific problem
        BF+= InnerProduct(psi,v[2])*dx
        BF+= InnerProduct(u[2], vphi)*dx
        BF+= -InnerProduct(exp(psi),vphi)*dx
        # the right hand sideBF
        # BF += -alpha * force[2] * v[2] * dx # force is 0 
        BF += -(psik)*v[2]*dx
        BF += -(obs)*vphi*dx
        return BF
    






In [11]:
""" WORKING PARAMETERS FOR SIMPLEST BONDED TEST"""
### 1 PHI = -Z
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 25

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*40 ,softening_n))

### 2

In [ ]:
max_PG_it = 20
max_iter_newton = 40
newton_damp = 0.3
softening_n = 30

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*50,softening_n))


for num, load in enumerate(gammas):
    G.Set(load) #incremental softening
    if num == softening_n-1:
        max_PG_it = 30
        tol_newton = 1e-8
        tol_delta = 1e-8
    for k in range(1,max_PG_it):
        print("PG it:",k)
        alpha.Set(1)# set to 1 for now
        psik.vec.data = psih.vec
        if k != 1:
            uk.vec.data = gfu.components[0].vec # update u_k to compare later

        G_funs = u,v,alpha,psi,psik 

        with TaskManager():
            BF = Assemble_BF(BF,G_funs,form) # get bi(not)linear form
            res = gfu.vec.CreateVector()
            w = gfu.vec.CreateVector()
        #iterations of newton for the NL al each PG step
        print("Solving nonlinear subproblem")
        for iter in range(max_iter_newton):
            with TaskManager():
                BF.Apply(gfu.vec,res)
                BF.AssembleLinearization(gfu.vec)
                inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                w.data = newton_damp * inv * res
                gfu.vec.data += -w
                delt = InnerProduct(w,res)
            print("Newton it:",iter, "Residual:", delt)
            if abs(delt) < tol_newton:
                print("Tol achieved",delt )
                break
        
            # update psik
        psik.vec.data = gfu.components[1].vec
        dif = Integrate((uk-gfu.components[0])*(uk-gfu.components[0]),mesh)
        print("U_k difference", dif)
        if dif < tol_delta:
            print("Tol PG achieved", dif)
            break
    Draw(gfu.components[0],mesh)



PG it: 1
Solving nonlinear subproblem
Newton it: 0 Residual: -23.61591876620819
Newton it: 1 Residual: -15.941702127085843
Newton it: 2 Residual: -9.695533175907677
Newton it: 3 Residual: -5.46426500537809
Newton it: 4 Residual: -2.9503874203461677
Newton it: 5 Residual: -1.549410015807334
Newton it: 6 Residual: -0.7978461377176815
Newton it: 7 Residual: -0.40502968434295006
Newton it: 8 Residual: -0.20350693160216174
Newton it: 9 Residual: -0.10149879723571303
Newton it: 10 Residual: -0.05035691460119827
Newton it: 11 Residual: -0.02489109939846014
Newton it: 12 Residual: -0.012271406337841233
Newton it: 13 Residual: -0.00603877126141078
Newton it: 14 Residual: -0.0029678722124252704
Newton it: 15 Residual: -0.0014573083555492632
Newton it: 16 Residual: -0.0007151292127638696
Newton it: 17 Residual: -0.00035077319953537496
Newton it: 18 Residual: -0.0001720023976859984
Newton it: 19 Residual: -8.432356668849397e-05
Newton it: 20 Residual: -4.1333093065681686e-05
Newton it: 21 Residual

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Solving nonlinear subproblem
Newton it: 0 Residual: 835.6047377189557
Newton it: 1 Residual: 570.0602219592037
Newton it: 2 Residual: 342.16253999155055
Newton it: 3 Residual: 190.6425989054912
Newton it: 4 Residual: 101.65981056526437
Newton it: 5 Residual: 52.738611698404114
Newton it: 6 Residual: 26.871001664127203
Newton it: 7 Residual: 13.526433761526722
Newton it: 8 Residual: 6.753019404154711
Newton it: 9 Residual: 3.352311770638026
Newton it: 10 Residual: 1.6576051989449707
Newton it: 11 Residual: 0.8173894571520461
Newton it: 12 Residual: 0.4022984996587414
Newton it: 13 Residual: 0.1977376757104622
Newton it: 14 Residual: 0.09710158416845681
Newton it: 15 Residual: 0.04765194810449019
Newton it: 16 Residual: 0.023374233696817733
Newton it: 17 Residual: 0.011461879600370482
Newton it: 18 Residual: 0.005619239661592128
Newton it: 19 Residual: 0.0027544288739744984
Newton it: 20 Residual: 0.001350013724025776
Newton it: 21 Residual: 0.0006616245909734351
Newton it: 22 R

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Solving nonlinear subproblem
Newton it: 0 Residual: 1608.7762038697722
Newton it: 1 Residual: 1093.796441867544
Newton it: 2 Residual: 655.000406042648
Newton it: 3 Residual: 364.4274909927281
Newton it: 4 Residual: 194.16210407338104
Newton it: 5 Residual: 100.67269718941789
Newton it: 6 Residual: 51.27683274099145
Newton it: 7 Residual: 25.80643766581637
Newton it: 8 Residual: 12.882000874908726
Newton it: 9 Residual: 6.394274477807871
Newton it: 10 Residual: 3.1615705435099772
Newton it: 11 Residual: 1.5589575036944812
Newton it: 12 Residual: 0.7672601811966233
Newton it: 13 Residual: 0.37711715053822864
Newton it: 14 Residual: 0.18518600624736253
Newton it: 15 Residual: 0.09087806878753979
Newton it: 16 Residual: 0.04457726914062387
Newton it: 17 Residual: 0.021859000274903843
Newton it: 18 Residual: 0.010716448530755634
Newton it: 19 Residual: 0.005252960156041166
Newton it: 20 Residual: 0.0025746024755637187
Newton it: 21 Residual: 0.0012617788896734298
Newton it: 22 Res

...

PG it: 1
Solving nonlinear subproblem
Newton it: 0 Residual: 3819.7343231315117
Newton it: 1 Residual: 2568.3515183490413
Newton it: 2 Residual: 1526.326967082681
Newton it: 3 Residual: 845.2083795056345
Newton it: 4 Residual: 449.0135885534191
Newton it: 5 Residual: 232.39664796797288
Newton it: 6 Residual: 118.23749538505163
Newton it: 7 Residual: 59.46471579822353
Newton it: 8 Residual: 29.670562918580984
Newton it: 9 Residual: 14.72369074293086
Newton it: 10 Residual: 7.278739496998628
Newton it: 11 Residual: 3.5887505069694847
Newton it: 12 Residual: 1.7661368195681046
Newton it: 13 Residual: 0.8680425069378165
Newton it: 14 Residual: 0.42624782619912144
Newton it: 15 Residual: 0.20917330081407914
Newton it: 16 Residual: 0.10260207831360929
Newton it: 17 Residual: 0.050311821679772765
Newton it: 18 Residual: 0.02466542769686273
Newton it: 19 Residual: 0.01209039621242204
Newton it: 20 Residual: 0.005925782293688886
Newton it: 21 Residual: 0.0029041439240960836
Newton it: 22 Residu

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Solving nonlinear subproblem
Newton it: 0 Residual: 4514.76656775458
Newton it: 1 Residual: 3023.4420284680223
Newton it: 2 Residual: 1791.7469808167048
Newton it: 3 Residual: 990.4456764236143
Newton it: 4 Residual: 525.6027389860326
Newton it: 5 Residual: 271.8565761526899
Newton it: 6 Residual: 138.25745870003874
Newton it: 7 Residual: 69.51621398163739
Newton it: 8 Residual: 34.68091260919944
Newton it: 9 Residual: 17.20867923270112
Newton it: 10 Residual: 8.506881534119303
Newton it: 11 Residual: 4.194215927192268
Newton it: 12 Residual: 2.0640986129392402
Newton it: 13 Residual: 1.014490794768499
Newton it: 14 Residual: 0.49816267947022214
Newton it: 15 Residual: 0.2444653128611032
Newton it: 16 Residual: 0.11991372643333864
Newton it: 17 Residual: 0.05880092968139642
Newton it: 18 Residual: 0.0288272943937728
Newton it: 19 Residual: 0.014130468878984814
Newton it: 20 Residual: 0.006925678402457495
Newton it: 21 Residual: 0.0033941824948570606
Newton it: 22 Residual: 0.0

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Solving nonlinear subproblem
Newton it: 0 Residual: 5185.503622064093
Newton it: 1 Residual: 3457.9205491464645
Newton it: 2 Residual: 2043.1697419786487
Newton it: 3 Residual: 1127.3281984213659
Newton it: 4 Residual: 597.558325212953
Newton it: 5 Residual: 308.85810935054764
Newton it: 6 Residual: 157.00914166627757
Newton it: 7 Residual: 78.92537249752513
Newton it: 8 Residual: 39.3699794049134
Newton it: 9 Residual: 19.53433239020747
Newton it: 10 Residual: 9.656450819015094
Newton it: 11 Residual: 4.761073539932283
Newton it: 12 Residual: 2.3431292978296363
Newton it: 13 Residual: 1.1516661427380008
Newton it: 14 Residual: 0.5655373572532681
Newton it: 15 Residual: 0.2775345677851971
Newton it: 16 Residual: 0.13613708014196835
Newton it: 17 Residual: 0.06675711905456924
Newton it: 18 Residual: 0.03272816290348021
Newton it: 19 Residual: 0.016042701076655237
Newton it: 20 Residual: 0.007862950209986749
Newton it: 21 Residual: 0.0038535413783360524
Newton it: 22 Residual: 0

In [9]:
# DO VTK    
vtk = VTKOutput(ma=mesh,coefs=[gfu.components[0],gfu.components[1]],names=["u","phi"],filename="gel")

vtk.Do()

'gel'